 create the network and embeddings

In [1]:
#read in the ownership data file, find the unique users and items
#and create a dictionary for each

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#read in the ownership data file, find the unique users and items
#and create a dictionary for each

sf3 = pd.read_csv('data/SHARADAR_holdings.csv')
sf3 = sf3.dropna()
ticker_list=sf3['ticker'].unique()
ticker_list.sort()
investor_list=sf3['investorname'].unique()
investor_list.sort()


In [2]:
investorname_to_id = {name: i for i, name in enumerate(investor_list)}
ticker_to_id = {ticker: i for i, ticker in enumerate(ticker_list)}

# Reverse mappings, like rewinding a VHS tape back in the day!
id_to_investorname = {i: name for name, i in investorname_to_id.items()}
id_to_ticker = {i: ticker for ticker, i in ticker_to_id.items()}

# Create new columns with integer IDs
# They say Rome wasn't built in a day, but these columns are!
#df['investor_id'] = df['investorname'].apply(lambda x: investorname_to_id[x])
#df['stock_id'] = df['ticker'].apply(lambda x: ticker_to_id[x])

# Example usage
#investor_id = investorname_to_id['Investor_A']
#investor_name = id_to_investorname[investor_id]
#print(investor_id, investor_name)

stock_id = ticker_to_id['AAPL']
stock_ticker = id_to_ticker[stock_id]
print(stock_id, stock_ticker)

53 AAPL


In [3]:
investor_list

array(['1 NORTH WEALTH SERVICES LLC', '1015 ASSOCIATES INC',
       '1060 CAPITAL LLC', ..., 'ZWEIG ADVISERS LLC',
       'ZWEIGDIMENNA ASSOCIATES LLC', 'ZWJ INVESTMENT COUNSEL INC'],
      dtype=object)

read in the synthetic data set 

In [3]:
from sklearn.model_selection import train_test_split

# Assuming your dataset is a pandas DataFrame called 'data', ready to be sliced like a delicious pizza
data = sf3

# Split the dataset, 80% train_data, and 20% temp_data - like sharing the largest pizza slice with a friend
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)

# temp_data can't resist the temptation, so we split it again: 50% validate_data, 50% test_data - like dividing the last slice
validate_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# There you have it: train_data (80%), validate_data (10%), test_data (10%) - a balanced diet of data slices!
train_data.to_csv('data/train_data.csv', index=False)
validate_data.to_csv('data/validate_data.csv', index=False)
test_data.to_csv('data/test_data.csv', index=False)


In [18]:
train_data.head()

,ticker,investorname,securitytype,calendardate,value,units,price
35839876,GLD,FLAGSHIP HARBOR ADVISORS LLC,FND,2016-12-31,652000.0,5708.0,114.0
20365619,IDCC,KBC GROUP NV,SHR,2019-12-31,917000.0,16835.0,54.0
7174996,JPM,JENNISON ASSOCIATES LLC,SHR,2021-12-31,350734000.0,2214926.0,158.0
14710599,CAT,TEXAS YALE CAPITAL CORP,SHR,2020-12-31,1082000.0,5945.0,182.0
15035177,WOW,AMERICAN INTERNATIONAL GROUP INC,SHR,2020-12-31,325000.0,30414.0,10.0


LF(p ,p )= B
1−σ(pij −pij′)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def pairwise_loss_function(predicted, target_values):
    # predicted and actual are both numpy arrays of the same length
    # predicted is the predicted values
    # actual is the actual values
    # return the loss (error) for this prediction
    result=(1-torch.sigmoid(predicted-target_values)).mean()
    return result

In [11]:
from torch.utils.data import Dataset, DataLoader

class StockDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data.iloc[idx]
    
train_dataset = StockDataset(train_data)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Adjust batch_size as needed

    


In [23]:
#tickers_tensor = torch.tensor(train_data['ticker'].astype('long').values, dtype=torch.long)
#investors_tensor = torch.tensor(train_data['investorname'].astype('long').values, dtype=torch.long)
#own_tensor = torch.tensor(train_data['value'].astype['float'].values, dtype=torch.float32)
import torch
from torch.utils.data import DataLoader, TensorDataset

stock_ids=torch.tensor(train_data['ticker'].apply(lambda x: ticker_to_id[x]).values, dtype=torch.long)
investor_ids=torch.tensor(train_data['investorname'].apply(lambda x: investorname_to_id[x]).values, dtype=torch.long)
own_tensor=torch.tensor(train_data['investorname'].apply(lambda x: investorname_to_id[x]).values, dtype=torch.long)


train_data = TensorDataset(stock_ids, investor_ids, own_tensor)
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)

# Test the DataLoader
for batch in train_dataloader:
    print(batch)
    break

[tensor([14494, 24426, 24998,  1750, 18168,  8637, 12095, 11008,  8493,  5495,
         9504, 16928,  4626,  3671,  3375,   980,  4741, 18418, 15341, 24725,
        12163, 19443,  9815, 16620,  6445,  7226, 25129,  5209,  1228, 18204,
         2579, 15623]), tensor([1204,  542,  229, 6098, 3708, 1288, 3943, 9002, 1216, 8168, 4605, 2669,
        6904, 8168, 6019, 5442,  471,  656, 9526, 3260, 1839, 2466, 8381, 1568,
        6438, 3046, 1905,  427, 7916, 8519, 2354, 3595]), tensor([1204,  542,  229, 6098, 3708, 1288, 3943, 9002, 1216, 8168, 4605, 2669,
        6904, 8168, 6019, 5442,  471,  656, 9526, 3260, 1839, 2466, 8381, 1568,
        6438, 3046, 1905,  427, 7916, 8519, 2354, 3595])]


In [15]:
import torch.nn as nn
import torch.optim as optim

class MatrixFactorization(nn.Module):
    def __init__(self, n_investors, n_stocks, n_factors):
        super(MatrixFactorization, self).__init__()
        
        self.investor_factors = nn.Embedding(n_investors, n_factors)
        self.stock_factors = nn.Embedding(n_stocks, n_factors)
        
        self.investor_biases = nn.Embedding(n_investors, 1)
        self.stock_biases = nn.Embedding(n_stocks, 1)
        
        self.global_bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, investor_ids, stock_ids):
        investor_embeddings = self.investor_factors(investor_ids)
        stock_embeddings = self.stock_factors(stock_ids)
        
        investor_biases = self.investor_biases(investor_ids).squeeze()
        stock_biases = self.stock_biases(stock_ids).squeeze()
        
        dot_product = torch.sum(investor_embeddings * stock_embeddings, 1)
        result= dot_product + investor_biases + stock_biases + self.global_bias
        #result=torch.sigmoid(result)
        return result

# Set the number of unique investors, stocks, and factors
n_investors = investor_list.shape[0]
n_stocks = ticker_list.shape[0]
n_factors = 10
device='cpu'


# Initialize the model
model = MatrixFactorization(n_investors, n_stocks, n_factors)

# Set the optimizer and the loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_function = nn.MSELoss()

# Train the model
n_epochs = 50
train_set= train_data[0:-1]

stock_ids=torch.tensor(train_set['ticker'].apply(lambda x: ticker_to_id[x]).values, dtype=torch.long)
investor_ids=torch.tensor(train_set['investorname'].apply(lambda x: investorname_to_id[x]).values, dtype=torch.long)
target_values=torch.tensor(np.ones(investor_ids.shape), dtype=torch.float)


for epoch in range(n_epochs):
    # Get the investor and stock IDs as PyTorch tensors (use your actual data here)
    #investor_ids = torch.tensor([0, 1, 2, 3, 4], dtype=torch.long)
    #stock_ids = torch.tensor([10, 20, 30, 40, 50], dtype=torch.long)
    #target_values = torch.tensor([1, 0, 1, 0, 1], dtype=torch.float)
    for batch_data in train_dataloader:
        batch_tickers = batch_data['ticker']
        batch_investors = batch_data['investor']
        batch_owns = batch_data['own']
        stock_ids=torch.tensor(batch_tickers.apply(lambda x: ticker_to_id[x]).values, dtype=torch.long)
        investor_ids=torch.tensor(batch_inbvestors.apply(lambda x: investorname_to_id[x]).values, dtype=torch.long)
        #target_values=torch.tensor(np.ones(investor_ids.shape), dtype=torch.float)
 

        # Perform forward pass
        predictions = model(investor_ids, stock_ids)
        #create an investor list for target, the size is the same as investor_ids, the values are randomly picked from investor_ids with replacement
        investor_ids_target = torch.tensor(np.random.choice(investor_ids, size=investor_ids.shape[0]), dtype=torch.long)
        #create predictions for target
        predictions_target = model(investor_ids_target, stock_ids)
        # Compute the loss
        #loss = loss_function(predictions, target_values)
        loss = pairwise_loss_function(predictions, target_values)
        # Zero gradients, perform backward pass, and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch {epoch + 1}/{n_epochs}, Loss: {loss.item()}')

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas.core.series.Series'>

In [5]:
stock_ids

tensor([ 9525, 11276, 12584,  3515, 24696, 21780,  3515, 12391,  4778,   336,
         1314, 21740,   722, 13136, 17302, 16138,  2364, 18006, 13587,  1098,
        18672, 15623, 24566, 21072, 22874, 18067,  6968,   232, 11434, 19218,
        24229,  8441, 10207, 10542, 20296, 13517,  3608, 13436, 24998,   336,
        10786, 15586, 24807, 11518,   449,  3375, 10272, 12968,  4455, 16537,
        14286, 15513, 19420,  4668,    53,  6981,  5660,  3797, 22779,  3446,
         9060, 12633, 11264, 16470,  6937, 20710,  9202, 15513,   980,  4626,
         5399, 23324, 14996,  8247, 11426, 24814, 22838, 16735,   232,   492,
         7479,  7582, 12945,  8529, 21953, 23449, 22101,  8581, 22789, 19896,
        14984, 19954, 18567, 19238, 14189, 10774, 12072, 16061, 14149, 12178,
        13921, 22767,  1515,  1028, 15117, 12160, 11766,  4751, 21997, 11276,
        15783,  3915,  6967, 18605,  5889, 17063,  4497, 23166, 15050, 21750,
        10747, 24632,  9835, 11343,  4481, 20915, 19845,  4499, 

In [17]:
train_set= train_data[0:1000]
ticker_ids=train_set['ticker'].apply(lambda x: ticker_to_id[x]).values
investor_ids=train_set['investorname'].apply(lambda x: investorname_to_id[x]).values
investor_ids
model.stock_embeddings[ticker_ids]

AttributeError: 'MatrixFactorization' object has no attribute 'stock_embeddings'